# Naive Bayesian Classifier
### Q1. Bayes Rule을 이해하고 Naive  Bayes classifier가 사용하는 사후 확률 계산 과정을 서술하세요.

- Bayes Rule:   
$P(w_i|x) = \frac{P(x|w_i)|P(w_i)}{P(x)} = \frac{P(x|w_i) P(w_i)}{\Sigma_j P(x|w_j)P(w_j)}$
  -
  - $P(x|w_i)\text{: 사후 확률, posterior}\\
P(x|w_i) \text{: 가능도/우도, likelihood}\\
P(w_i) \text{: 사전 확률, prior}\\
P(x) \text{: 증거, evidence}$

A1.  각 클래스 w_i에 대한 사전확률 P(w_i)를 각 클래스 빈도를 통해 계산한다. 각 특성 x_j가 독립적이므로 가능도 P(x|w_i)는 각 특성의 조건부확률의 곱으로 구할 수 있다. 베이즈 정리를 사용해서 사후 확률 P(w_i|x)를 계산한다. 이 때 P(x)는 모든 클래스에 대한 P(x∣w_i)⋅P(w_i) 의 합으로 계산한다.

### Q2. Naive Bayes Classification 방법을 이용해서 다음 생성된 리뷰 데이터에 기반한 감정 분석을 해봅시다.

In [1]:
# pip install pandas
import pandas as pd
import re

In [4]:
# 리뷰 데이터 생성
data = {
    'review': [
        'I love this great product! It exceeded my expectations.',
        'The Worst purchase I have ever made. Completely useless.',
        'It is an average product, nothing special but not terrible either.',
        'Great service and who can help but love this design? Highly recommend!',
        'Terrible experience, I will never buy from this poor brand again.',
        'It’s acceptable, but I expected better service, not just an acceptable one.',
        'Absolutely wonderful! I am very satisfied with this great service.',
        'The quality is poor and it broke after one use. Terrible enough!',
        'Acceptable product for the price, but there are better options out there.',
        'Great quality and fast shipping with wonderful service! I love it'
    ],
    'sentiment': [
        'positive', 'negative', 'neutral', 'positive', 'negative',
        'neutral', 'positive', 'negative', 'neutral', 'positive',
    ]
}
df = pd.DataFrame(data)
df.head()

,review,sentiment
0,I love this great product! It exceeded my expe...,positive
1,The Worst purchase I have ever made. Completel...,negative
2,"It is an average product, nothing special but ...",neutral
3,Great service and who can help but love this d...,positive
4,"Terrible experience, I will never buy from thi...",negative


In [5]:
# 불용어 리스트 정의
stopwords = ['i', 'my', 'am', 'this', 'it', 'its', 'an', 'a', 'the', 'is', 'are', 'and', 'product', 'service']

In [8]:
# 텍스트 전처리 함수 정의
def preprocess_text(text):
    # 소문자로 변환
    text = text.lower()
    # 특수 기호 제거
    text = re.sub(r'[^a-z\s]', '', text)
    # 불용어 제거
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

# 모든 리뷰에 대해 전처리 수행
df['review'] = df['review'].apply(preprocess_text)

기본적인 데이터 전처리가 완료되었습니다!
이제부터 직접 나이브 베이지안 분류를 수행해 봅시다.  
우리가 분류하고자 하는 문장은 총 두가지 입니다.  
전처리가 완료되었다고 치고,   
첫번째 문장은 **'love, great, awesome'**,  
두번째 문장은 **'terrible, not, never'** 입니다. 

사전 확률 $P(positive), P(negative), P(neutral)$을 구합니다. 

In [9]:
# 사전 확률 구하는 코드를 작성해주세요.

# 전체 리뷰의 개수
total_reviews = len(df)

# 각 클래스별 리뷰의 개수
positive_reviews = len(df[df['sentiment'] == 'positive'])
negative_reviews = len(df[df['sentiment'] == 'negative'])
neutral_reviews = len(df[df['sentiment'] == 'neutral'])

# 사전 확률 계산
P_positive = positive_reviews / total_reviews
P_negative = negative_reviews / total_reviews
P_neutral = neutral_reviews / total_reviews

P_positive, P_negative, P_neutral

(0.4, 0.3, 0.3)

가능도를 구하기 위한 확률들을 계산합니다.  
예: 첫번째 문장 분류를 위해서는, $P(love|positive), P(great|positive), P(awesome|positive)\\
P(love|negative), P(great|negative), P(awesome|negative)\\
P(love|neutral), P(great|neutral), P(great|neutral)$를 구합니다.

이 때 CountVectorizer를 사용하여 도출한 단어 벡터를 활용하면 확률들을 간편하게 구할 수 있습니다.  
참고: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
review_array = vectorizer.fit_transform(df['review']).toarray()
review_array

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 2, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 

In [14]:
vectorizer.get_feature_names_out()

array(['absolutely', 'acceptable', 'after', 'again', 'average', 'better',
       'brand', 'broke', 'but', 'buy', 'can', 'completely', 'design',
       'either', 'enough', 'ever', 'exceeded', 'expectations', 'expected',
       'experience', 'fast', 'for', 'from', 'great', 'have', 'help',
       'highly', 'just', 'love', 'made', 'never', 'not', 'nothing', 'one',
       'options', 'out', 'poor', 'price', 'purchase', 'quality',
       'recommend', 'satisfied', 'shipping', 'special', 'terrible',
       'there', 'use', 'useless', 'very', 'who', 'will', 'with',
       'wonderful', 'worst'], dtype=object)

In [15]:
vectorizer.vocabulary_

{'love': 28,
 'great': 23,
 'exceeded': 16,
 'expectations': 17,
 'worst': 53,
 'purchase': 38,
 'have': 24,
 'ever': 15,
 'made': 29,
 'completely': 11,
 'useless': 47,
 'average': 4,
 'nothing': 32,
 'special': 43,
 'but': 8,
 'not': 31,
 'terrible': 44,
 'either': 13,
 'who': 49,
 'can': 10,
 'help': 25,
 'design': 12,
 'highly': 26,
 'recommend': 40,
 'experience': 19,
 'will': 50,
 'never': 30,
 'buy': 9,
 'from': 22,
 'poor': 36,
 'brand': 6,
 'again': 3,
 'acceptable': 1,
 'expected': 18,
 'better': 5,
 'just': 27,
 'one': 33,
 'absolutely': 0,
 'wonderful': 52,
 'very': 48,
 'satisfied': 41,
 'with': 51,
 'quality': 39,
 'broke': 7,
 'after': 2,
 'use': 46,
 'enough': 14,
 'for': 21,
 'price': 37,
 'there': 45,
 'options': 34,
 'out': 35,
 'fast': 20,
 'shipping': 42}

In [16]:
frequency_matrix = pd.DataFrame(review_array, columns = vectorizer.get_feature_names_out())
frequency_matrix = pd.concat([df['sentiment'], frequency_matrix], axis=1)
frequency_matrix

,sentiment,absolutely,acceptable,after,again,average,better,brand,broke,but,...,terrible,there,use,useless,very,who,will,with,wonderful,worst
0,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,negative,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,neutral,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,positive,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,negative,0,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
5,neutral,0,2,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,positive,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,0
7,negative,0,0,1,0,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0
8,neutral,0,1,0,0,0,1,0,0,1,...,0,2,0,0,0,0,0,0,0,0
9,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [18]:
# 위와 같이 조건부 확률을 구하는 코드를 작성해주세요

def calculate_conditional_probability(frequency_matrix, target_word, sentiment):
    if target_word in frequency_matrix.columns:
        word_count_in_class = frequency_matrix[frequency_matrix['sentiment'] == sentiment][target_word].sum()
    else:
        word_count_in_class = 0  # 단어가 존재하지 않으면 카운트를 0으로 설정
    
    total_words_in_class = frequency_matrix[frequency_matrix['sentiment'] == sentiment].iloc[:, 1:].sum().sum()
    total_unique_words = len(vectorizer.get_feature_names_out())
    
    # 라플라스 스무딩 적용
    return (word_count_in_class + 1) / (total_words_in_class + total_unique_words)

words = ['love', 'great', 'awesome']

# 첫번째 문장의 각 단어에 대한 조건부 확률 계산
for word in words:
    print(f"P({word}|positive): {calculate_conditional_probability(frequency_matrix, word, 'positive')}")
    print(f"P({word}|negative): {calculate_conditional_probability(frequency_matrix, word, 'negative')}")
    print(f"P({word}|neutral): {calculate_conditional_probability(frequency_matrix, word, 'neutral')}")


P(love|positive): 0.05
P(love|negative): 0.01282051282051282
P(love|neutral): 0.01282051282051282
P(great|positive): 0.0625
P(great|negative): 0.01282051282051282
P(great|neutral): 0.01282051282051282
P(awesome|positive): 0.0125
P(awesome|negative): 0.01282051282051282
P(awesome|neutral): 0.01282051282051282


독립성 가정을 이용하여 가능도(likelihood)를 구합니다.  
첫번째 문장 예시: $P(love, great, awesome|positive), P(love, great, awesome|negative), P(love, great, awesome|neutral)$

In [20]:
# 가능도 구하는 코드를 작성해주세요.

target_review1 = ['love', 'great', 'awesome']
target_review2 = ['terrible', 'not', 'never']

# 첫 번째 문장의 조건부 확률 계산
likelihoods_positive_review1 = [calculate_conditional_probability(frequency_matrix, word, 'positive') for word in target_review1]
likelihoods_negative_review1 = [calculate_conditional_probability(frequency_matrix, word, 'negative') for word in target_review1]
likelihoods_neutral_review1 = [calculate_conditional_probability(frequency_matrix, word, 'neutral') for word in target_review1]

# 두 번째 문장의 조건부 확률 계산
likelihoods_positive_review2 = [calculate_conditional_probability(frequency_matrix, word, 'positive') for word in target_review2]
likelihoods_negative_review2 = [calculate_conditional_probability(frequency_matrix, word, 'negative') for word in target_review2]
likelihoods_neutral_review2 = [calculate_conditional_probability(frequency_matrix, word, 'neutral') for word in target_review2]

# 첫 번째 문장의 각 감정에 대한 조건부 확률 출력
print("target_review1")
print(f"Likelihoods for 'positive': {likelihoods_positive_review1}")
print(f"Likelihoods for 'negative': {likelihoods_negative_review1}")
print(f"Likelihoods for 'neutral': {likelihoods_neutral_review1}")

# 두 번째 문장의 각 감정에 대한 조건부 확률 출력
print("\ntarget_review2")
print(f"Likelihoods for 'positive': {likelihoods_positive_review2}")
print(f"Likelihoods for 'negative': {likelihoods_negative_review2}")
print(f"Likelihoods for 'neutral': {likelihoods_neutral_review2}")


target_review1
Likelihoods for 'positive': [0.05, 0.0625, 0.0125]
Likelihoods for 'negative': [0.01282051282051282, 0.01282051282051282, 0.01282051282051282]
Likelihoods for 'neutral': [0.01282051282051282, 0.01282051282051282, 0.01282051282051282]

target_review2
Likelihoods for 'positive': [0.0125, 0.0125, 0.0125]
Likelihoods for 'negative': [0.038461538461538464, 0.01282051282051282, 0.02564102564102564]
Likelihoods for 'neutral': [0.02564102564102564, 0.038461538461538464, 0.01282051282051282]


위에서 구한 사전 확률과 가능도를 이용하여 타겟 문장이 positive, negative, neutral일 확률을 구하고 최종적으로 어떤 감성일지 분석해봅니다.

In [21]:
import numpy as np
# 최종 확률 구하는 코드를 작성해주세요.

# 사전 확률
P_positive = positive_reviews / total_reviews
P_negative = negative_reviews / total_reviews
P_neutral = neutral_reviews / total_reviews

# 첫 번째 문장에 대한 최종 확률 계산
P_positive_given_review1 = np.prod(likelihoods_positive_review1) * P_positive
P_negative_given_review1 = np.prod(likelihoods_negative_review1) * P_negative
P_neutral_given_review1 = np.prod(likelihoods_neutral_review1) * P_neutral

# 두 번째 문장에 대한 최종 확률 계산
P_positive_given_review2 = np.prod(likelihoods_positive_review2) * P_positive
P_negative_given_review2 = np.prod(likelihoods_negative_review2) * P_negative
P_neutral_given_review2 = np.prod(likelihoods_neutral_review2) * P_neutral

# 결과 출력
print(f"P(positive|target_review1): {P_positive_given_review1}")
print(f"P(negative|target_review1): {P_negative_given_review1}")
print(f"P(neutral|target_review1): {P_neutral_given_review1}")

print(f"\nP(positive|target_review2): {P_positive_given_review2}")
print(f"P(negative|target_review2): {P_negative_given_review2}")
print(f"P(neutral|target_review2): {P_neutral_given_review2}")


P(positive|target_review1): 1.5625000000000004e-05
P(negative|target_review1): 6.321751883882061e-07
P(neutral|target_review1): 6.321751883882061e-07

P(positive|target_review2): 7.812500000000002e-07
P(negative|target_review2): 3.7930511303292367e-06
P(neutral|target_review2): 3.7930511303292367e-06


A2-1.   
Target review1의 분류 결과: positive  
Target review2의 분류 결과: negative, neutral

Q2-2. 나이브 베이지안 기반 확률을 구하는 과정에서 어떤 문제점을 발견할 수 있었나요? 그리고 그 문제를 해결하기 위한 방법에 대해 간략하게 조사 및 서술해 주세요. (힌트: Laplace smoothing)

A2-2. 나이브 베이즈 분류기를 사용할 때 가장 큰 문제 중 하나는 특정 단어가 특정 클래스에서 한 번도 등장하지 않았을 때 발생합니다. 이로 인해 조건부 확률이 0이 되면, 이 확률을 기반으로 한 최종 계산에서 전체 확률이 0이 되는 현상이 발생할 수 있습니다.
라플라스 스무딩은 이러한 제로 확률 문제를 해결하기 위해 모든 가능한 단어가 학습 데이터에 적어도 한 번은 등장한 것처럼 가정하는 기법입니다. 즉, 각 단어의 빈도수에 1을 더해주고, 전체 단어 수에도 적절한 값을 더하여 확률을 계산합니다.